# **PCA_LDA**

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

# Generate the Data Matrix and the Label vector

In [22]:
paths=['datasets/s'+str(i) for i in range(1,41)]
cnt=0
Data=np.zeros((400,10304))
labels=np.zeros((400,1))
for i in range(40):
    labels[i*10:(i+1)*10]=i+1  
for path in paths:
    files = os.listdir(path)
    for file in files:
        img = Image.open(path + "/" + file)
        np_img = np.array(img)
        np_img = np_img.flatten()
        Data[cnt]=np_img
        cnt+=1
print(labels)        
            

[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 2.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 3.]
 [ 4.]
 [ 4.]
 [ 4.]
 [ 4.]
 [ 4.]
 [ 4.]
 [ 4.]
 [ 4.]
 [ 4.]
 [ 4.]
 [ 5.]
 [ 5.]
 [ 5.]
 [ 5.]
 [ 5.]
 [ 5.]
 [ 5.]
 [ 5.]
 [ 5.]
 [ 5.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 6.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 7.]
 [ 8.]
 [ 8.]
 [ 8.]
 [ 8.]
 [ 8.]
 [ 8.]
 [ 8.]
 [ 8.]
 [ 8.]
 [ 8.]
 [ 9.]
 [ 9.]
 [ 9.]
 [ 9.]
 [ 9.]
 [ 9.]
 [ 9.]
 [ 9.]
 [ 9.]
 [ 9.]
 [10.]
 [10.]
 [10.]
 [10.]
 [10.]
 [10.]
 [10.]
 [10.]
 [10.]
 [10.]
 [11.]
 [11.]
 [11.]
 [11.]
 [11.]
 [11.]
 [11.]
 [11.]
 [11.]
 [11.]
 [12.]
 [12.]
 [12.]
 [12.]
 [12.]
 [12.]
 [12.]
 [12.]
 [12.]
 [12.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [13.]
 [14.]
 [14.]
 [14.]
 [14.]
 [14.]
 [14.]
 [14.]
 [14.]
 [14.]
 [14.]
 [15.]
 [15.]
 [15.]

In [ ]:
image_height = 112
image_width = 92
for i in range (10):
 image_array = np.reshape(Data[i*10], (image_height, image_width))

# Plot the image
 plt.imshow(image_array, cmap='gray')  # Use cmap='gray' for grayscale images
 plt.axis('off')  # Turn off axis
 plt.show()

# Splitting The Data Set

In [29]:
X_train=Data[1::2]
X_test=Data[::2]
y_train=labels[1::2]
y_test=labels[::2]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(200, 10304)
(200, 10304)
(200, 1)
(200, 1)
